### <font color='blue'> Progama para Hacer el Rol de Servicios

#### <font color='black'> Librerias requeridas

In [740]:
from calendar import monthrange
from datetime import datetime
import pandas as pd
import locale, calendar
from stat import S_IREAD, S_IRGRP, S_IROTH
from styleframe import StyleFrame, Styler, utils
import shutil, queue, yaml

#### <font color='blue'> Cargar el archivo con nombres de los oficiales y fechas de comisiones y svs. ctl.

In [776]:
with open("Relacion Oficiales.yaml","r") as s:
    y=yaml.safe_load(s)
nombreOficiales=[Oficial['Nombre'] for Oficial in y['Oficiales']]
totalOfs=len(nombresOficiales)

#### <font color='blue'> Establecer la fecha del Rol, y crear la tabla preliminar

In [777]:
mes=11 
año=2020

locale.setlocale(locale.LC_ALL, 'es_ES') #idioma Español
diasMes=[datetime(año, mes, x+1).strftime("%d\n%a") for x in range(monthrange(año,mes)[1])]
df=pd.DataFrame(columns=diasMes)
df.insert(0, column='Oficiales', value=nombresOficiales)
df=colocar_comisiones(df,y).fillna("")
df

,Oficiales,01\ndo.,02\nlu.,03\nma.,04\nmi.,05\nju.,06\nvi.,07\nsá.,08\ndo.,09\nlu.,...,21\nsá.,22\ndo.,23\nlu.,24\nma.,25\nmi.,26\nju.,27\nvi.,28\nsá.,29\ndo.,30\nlu.
0,Cap. 1/o. I.C.E. Jose Luis Revilla Manzur,,,,,,,,,,...,,,,,,,,,,
1,Cap. 1/o. I.C.E. Jaime Mendoza Soto,,CTL,,,,,,,,...,,,,,,,,,,
2,Cap. 1/o. I.C.E. Carlos Mendoza Sanchez,,,,,,,CTL,,,...,,,,,,,,,,
3,Tte. Trans. Luis Raymundo de Jesus Mixteco,C,C,C,C,C,C,,,CTL,...,,,,,,,,,,
4,Tte. Rep. Jose Luis Felix,,,,,,,,,,...,,,,,,,,,,
5,Cap. 1/o. I.C.E. Fernando Alcantar Hernandez,,,,,,,,,,...,,,,,,,,,,
6,Cap. 1/o. I.C.E. Rafael Espinoza Reyes,C,,,CTL,,,,,,...,,,,,,,,,,


#### <font color='blue'> Asignar comisiones

### <font color='red'> Algoritmo para designar los servicios

#### <font color='bluw'> Crear colas/queue

In [743]:
serviciosSD=[]
for index,Oficial in enumerate(y['Oficiales']):
    serviciosSD.append([Oficial['Servicios del mes anterior fin de semana'],len(y['Oficiales'])-index,Oficial['Nombre']])
serviciosSD.sort()

serviciosLV=[]
for index,Oficial in enumerate(y['Oficiales']):
    serviciosLV.append([Oficial['Servicios del mes anterior entresemana'],index,Oficial['Nombre']])
serviciosLV.sort()


#### <font color='bluw'> Designar fin de semana

In [744]:
dias = len(df.columns)

for dia in range(1,dias): #avanzamos un dia hasta acompletar el mes
    # Caso en el que es fin de semana (queue fin semana):
    
    if datetime(año, mes, dia).strftime("%A") == 'sábado' or datetime(año, mes, dia).strftime("%A") == 'domingo': 
        if VerCtl(df,dia)[0]==False: # (¿Hay alguien de cuartel?)
            serviciosSD.sort()
            Of_designado=serviciosSD.pop(0) # 1er Propuesta para el sv.
            #¿Esta de comision? ¿Estuvo de servicio de Ctl ayer? ¿Estuvo ayer de otro sv?
            if df.loc[df['Oficiales']==Of_designado[2],df.columns[dia]].item() == "" and df.loc[df['Oficiales']==Of_designado[2],df.columns[dia-1]].item() != 'CTL' and (df.loc[df['Oficiales']==Of_designado[2],df.columns[dia-1]].item() == "" or df.loc[df['Oficiales']==Of_designado[2],df.columns[dia-1]].item() == "C"):
                df.loc[df['Oficiales']==Of_designado[2],df.columns[dia]] = Of_designado[0] + 1   # Se coloca en el rol, el sv a la persona designada, con anotacion de los números de servicios que lleva
                serviciosSD.append([Of_designado[0]+1,Of_designado[1],Of_designado[2]]) # Se introduce al primer propuesto a la cola + 1 sv
            #si el oficial designado no puede montar, nos vamos por el que sigue
            else:
                Of_designado2=serviciosSD.pop(0)
                df.loc[df['Oficiales']==Of_designado2[2],df.columns[dia]] = Of_designado2[0] + 1   # Se coloca en el rol, el sv a la persona designada, con anotacion de los números de servicios que lleva
                serviciosSD.append([Of_designado2[0]+1,Of_designado2[1],Of_designado2[2]]) # Se introduce al 2o propuesto a la cola + 1 sv
                serviciosSD.append([Of_designado[0],Of_designado[1],Of_designado[2]]) # Se introduce al 1er propuesto a la cola sv
        elif VerCtl(df,dia)[0]==True: # Si ya habia alguien de Ctl, le contamos su servicio
            OfCtl=VerCtl(df,dia)[1]
            indexCtl=elementInSubarray(serviciosSD, OfCtl)
            serviciosSD[indexCtl][0]+=1
        serviciosSD.sort()

        
     # Caso en el que es entre semana (queue entre semana):
    elif datetime(año, mes, dia).strftime("%A") != 'sábado' and datetime(año, mes, dia).strftime("%A") != 'domingo': 
        if VerCtl(df,dia)[0]==False: # (¿Hay alguien de cuartel?)
            serviciosLV.sort()
            Of_designado=serviciosLV.pop(0) # 1er Propuesta para el sv.
            #¿Esta de comision? ¿Estuvo de servicio de Ctl ayer? ¿Estuvo ayer de otro sv?
            if Of_designado[2]=='Cap. 1/o. I.C.E.Berenice Flores Mancilla':
                print(df.loc[df['Oficiales']==Of_designado[2],df.columns[dia]].item() )
                print(df.loc[df['Oficiales']==Of_designado[2],df.columns[dia-1]].item() )
                print(df.loc[df['Oficiales']==Of_designado[2],df.columns[dia-1]].item() )
                break
            if df.loc[df['Oficiales']==Of_designado[2],df.columns[dia]].item() == "" and df.loc[df['Oficiales']==Of_designado[2],df.columns[dia-1]].item() != 'CTL' and (df.loc[df['Oficiales']==Of_designado[2],df.columns[dia-1]].item() == "" or df.loc[df['Oficiales']==Of_designado[2],df.columns[dia-1]].item() == "C"):
                df.loc[df['Oficiales']==Of_designado[2],df.columns[dia]] = Of_designado[0] + 1   # Se coloca en el rol, el sv a la persona designada, con anotacion de los números de servicios que lleva
                serviciosLV.append([Of_designado[0]+1,Of_designado[1],Of_designado[2]]) # Se introduce al primer propuesto a la cola + 1 sv
            #si el oficial designado no puede montar, nos vamos por el que sigue
            else:
                Of_designado2=serviciosLV.pop(0)
                df.loc[df['Oficiales']==Of_designado2[2],df.columns[dia]] = Of_designado2[0] + 1   # Se coloca en el rol, el sv a la persona designada, con anotacion de los números de servicios que lleva
                serviciosLV.append([Of_designado2[0]+1,Of_designado2[1],Of_designado2[2]]) # Se introduce al 2o propuesto a la cola + 1 sv
                serviciosLV.append([Of_designado[0],Of_designado[1],Of_designado[2]]) # Se introduce al 1er propuesto a la cola sv
        elif VerCtl(df,dia)[0]==True: # Si ya habia alguien de Ctl, le contamos su servicio
            OfCtl=VerCtl(df,dia)[1]
            indexCtl=elementInSubarray(serviciosLV, OfCtl)
            serviciosLV[indexCtl][0]+=1
        serviciosLV.sort()

ValueError: can only convert an array of size 1 to a Python scalar

In [761]:
df.loc[df['Oficiales']==Of_designado[2],df.columns[dia]].item()

ValueError: can only convert an array of size 1 to a Python scalar

In [759]:
a

'Series([], Name: 03\\nma., dtype: object)'

In [766]:
df.iloc[0,:5]

Oficiales    Cap. 1/o. I.C.E. Jose Luis Revilla Manzur
01\ndo.                                               
02\nlu.                                               
03\nma.                                               
04\nmi.                                               
Name: 0, dtype: object

In [731]:
serviciosSD

[[0, 1, 'Cap. 1/o. I.C.E. Rafael Espinoza Reyes'],
 [0, 3, 'Tte. Rep. Jose Luis Felix'],
 [0, 4, 'Tte. Trans. Luis Raymundo de Jesus Mixteco'],
 [0, 5, 'Cap. 1/o. I.C.E. Carlos Mendoza Sanchez'],
 [0, 6, 'Cap. 1/o. I.C.E. Jaime Mendoza Soto'],
 [0, 7, 'Cap. 1/o. I.C.E.Berenice Flores Mancilla'],
 [1, 2, 'Cap. 1/o. I.C.E. Fernando Alcantar Hernandez']]

#### <font color='bluw'> Designar entresemana

In [565]:
dias = len(df.columns)
for dia in range(1,dias): #rotando filas del rol (dia uno)
    if VerCtl(df,dia)[0]==False: # (¿Hay alguien de cuartel?)
        Of_designado=serviciosLV.pop(0) #Se designa al primer propuesto 
        if df.loc[df['Oficiales']==Of_designado[2],df.columns[dia]] == None and df.loc[df['Oficiales']==Of_designado[2],df.columns[dia-1]] != 'CTL' and type(df.loc[df['Oficiales']==Of_designado[2],df.columns[dia-1]]) != int: #¿Esta de comision? ¿Estuvo de servicio ayer?
            df.loc[df['Oficiales']==Of_designado[2],df.columns[dia]] = Of_designado[0] + 1   # Se coloca en el rol, el sv a la persona designada, con anotacion de los números de servicios que lleva
            serviciosLV.append([Of_designado[0]+1,Of_designado[1],Of_designado[2]]) # Se introduce al primer propuesto a la cola + 1 sv

        else:
            Of_designado2=serviciosLV.pop(0)
            df.loc[df['Oficiales']==Of_designado2[2],df.columns[dia]] = Of_designado2[0] + 1   # Se coloca en el rol, el sv a la persona designada, con anotacion de los números de servicios que lleva
            serviciosLV.append([Of_designado2[0]+1,Of_designado2[1],Of_designado2[2]]) # Se introduce al 2o propuesto a la cola + 1 sv
            serviciosLV.append([Of_designado[0],Of_designado[1],Of_designado[2]]) # Se introduce al 1er propuesto a la cola sv

            
            
        
        #df.loc[df['Oficiales']==Of_designado[2],df.columns[dia]] = Of_designado[0] + 1   # Se coloca en el rol, el sv a la persona designada, con anotacion de los números de servicios que lleva
        #serviciosLV.append([Of_designado[0]+1,Of_designado[1],Of_designado[2]]) # Se introduce al primer propuesto a la cola + 1 sv
        serviciosLV.sort()
    elif VerCtl(df,dia)[0]==True:
        OfCtl=VerCtl(df,dia)[1]
        indexCtl=elementInSubarray(serviciosLV, OfCtl)
        serviciosLV[indexCtl][0]+=1
    serviciosLV.sort()
#        df.loc[df['Oficiales']==OfCtl,]


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [570]:
type(df.iloc[1,1])
#df.iloc[1,1]

float

#### <font color='blue'> Guardar los archivos en EXCEL

In [745]:
sf=StyleFrame(df)
sf.to_excel(excel_writer='Rol Dic.xlsx', best_fit=list(df.columns), index=False).save()

#### <font color='blue'> Funciones del Sistema

In [485]:
#funcion para verificar si no hay alguien de ctl
def VerCtl(df,dia):
    AlguienCtl=False
    for x in range(totalOfs):
        if df.iloc[x,dia]=='CTL':
            AlguienCtl=True
            nombreCtl=df.iloc[x,0]
            return [AlguienCtl,nombreCtl]
    return [AlguienCtl,None]

In [564]:
def elementInSubarray(array, string1):
    for x in array:
        if string1 in x:
            return array.index(x)
    
#elementInSubarray(serviciosLV,'Cap. 1/o. I.C.E. Jaime Mendoza Soto')

In [430]:
#Funcion para obtener la fecha para utilizarse en el .loc de acuerdo a las comisiones nombradas
def get_date(df,dia):
    for match in list(df.columns):
        if str(int(dia)).zfill(2) in match:
            return(match)

def colocar_comisiones (df,y):
    for Oficial in y['Oficiales']:
        #OficialObj = list(Oficial.values())[0]
        nombre= Oficial['Nombre']
        cuarteles= Oficial['Cuartel']
        comisiones= Oficial['Comisiones']
        vacaciones= Oficial['Vacaciones']
        for cuartel in cuarteles:
            df.loc[df['Oficiales']==nombre,get_date(df,cuartel)]='CTL'
        for comision in comisiones:
            df.loc[df['Oficiales']==nombre,get_date(df,comision)]='C'
        for vacacion in vacaciones:
            df.loc[df['Oficiales']==nombre,get_date(df,vacacion)]='A'
    return df    